In [2]:
from google.colab import files
uploaded = files.upload()

Saving ptbxl_database.csv to ptbxl_database.csv


In [3]:
from google.colab import files
uploaded = files.upload()

Saving scp_statements.csv to scp_statements.csv


In [4]:
from google.colab import files
uploaded = files.upload()

Saving Lab Session Data.xlsx to Lab Session Data.xlsx


In [5]:
from google.colab import files
uploaded = files.upload()

Saving 01000_lr.dat to 01000_lr.dat
Saving 01000_lr.hea to 01000_lr.hea
Saving 01001_lr.dat to 01001_lr.dat


In [25]:
#A7
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import ast

# Step 1: Load and filter ECG data
def load_ecg_data(path_ptb="ptbxl_database.csv", path_scp="scp_statements.csv"):
    df = pd.read_csv(path_ptb)
    scp = pd.read_csv(path_scp)
    diagnostic_codes = scp[scp['diagnostic'] == 1.0]['Unnamed: 0'].tolist()
    df['scp_codes'] = df['scp_codes'].apply(ast.literal_eval)
    df['diagnostic_codes'] = df['scp_codes'].apply(lambda x: [code for code in x if code in diagnostic_codes])
    mi_codes = ['IMI', 'ASMI', 'AMI', 'ALMI', 'ILMI']
    def assign_label(codes):
        if "NORM" in codes:
            return "NORM"
        elif any(code in mi_codes for code in codes):
            return "MI"
        return None
    df['label'] = df['diagnostic_codes'].apply(assign_label)
    df = df.dropna(subset=["label", "age", "weight"])
    return df[["age", "weight", "label"]]

# Step 2: Prepare training data
def prepare_data(df):
    X = df[["age", "weight"]].values
    y = LabelEncoder().fit_transform(df["label"].values)  # 0 = MI, 1 = NORM
    return X, y

# Step 3: Perform GridSearchCV
def tune_knn_hyperparameter(X, y):
    param_grid = {'n_neighbors': list(range(1, 21))}  # k from 1 to 20
    model = KNeighborsClassifier()
    grid = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')  # 5-fold cross-validation
    grid.fit(X, y)
    return grid

# Step 4: Display results
def run_a7():
    df = load_ecg_data()
    X, y = prepare_data(df)
    grid = tune_knn_hyperparameter(X, y)
    print(f"Best k: {grid.best_params_['n_neighbors']}")
    print(f"Best Accuracy: {grid.best_score_:.4f}")

# Run A7
run_a7()


Best k: 19
Best Accuracy: 0.8082
